<a href="https://colab.research.google.com/github/yk-Jeong/Kaggle-Studies/blob/main/9th_%EA%B0%9D%EC%B2%B4_%ED%83%90%EC%A7%80_(Object_Detection)_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 참조 : https://github.com/wikibook/dl-vision

# 객체 탐지 (Object Detection)

- 한 이미지에서 **객체**와 그 **경계 상자**(bounding box)를 탐지

- 객체 탐지 알고리즘은 일반적으로 *이미지를 입력*으로 받고, *경계 상자*와 *객체 클래스 리스트*를 출력

- 경계 상자에 대해 그에 대응하는 **예측 클래스**와 클래스의 **신뢰도**(confidence)를 출력

## Applications

- 자율 주행 자동차에서 다른 자동차와 보행자를 찾을 때

- 의료 분야에서 방사선 사진을 사용해 종양이나 위험한 조직을 찾을 때

- 제조업에서 조립 로봇이 제품을 조립하거나 수리할 때

- 보안 산업에서 위협을 탐지하거나 사람을 셀 때

## 용어 설명

### Bounding Box

- 이미지에서 하나의 객체 전체를 포함하는 가장 작은 직사각형

  <img src="https://miro.medium.com/max/850/1*KL6r494Eyfh3iYEXQA2tzg.png">

  <sub>[이미지 출처] https://medium.com/anolytics/how-bounding-box-annotation-helps-object-detection-in-machine-learning-use-cases-431d93e7b25b</sub>

### IOU(Intersection Over Union)
- 실측값(Ground Truth)과 모델이 예측한 값이 얼마나 겹치는지를 나타내는 지표

  <img src="https://pyimagesearch.com/wp-content/uploads/2016/09/iou_equation.png" width="300">

- IOU가 높을수록 잘 예측한 모델

  <img src="https://pyimagesearch.com/wp-content/uploads/2016/09/iou_examples.png" width="400">

<br>

- 예시
  <img src="https://www.pyimagesearch.com/wp-content/uploads/2016/09/iou_stop_sign.jpg">

  <sub>[이미지 출처] https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/</sub>

In [1]:
def compute_iou(pred_box, gt_box): #예측값과 실제 값을 입력받으면
  x1 = np.maximum(pred_box[0], gt_box[0])
  y1 = np.maximum(pred_box[1], gt_box[1])
  x2 = np.maximum(pred_box[2], gt_box[2])
  y2 = np.maximum(pred_box[3], gt_box[3])
  
  intersection = np.maximum(x2 - x1, 0) * np.maximum(y2 - y1, 0)

  pred_box_area = (pred_box[2] - pred_box[0]) * (pred_box[3] - pred_box[1])
  gt_box_area = (gt_box[2] - gt_box[0]) * (gt_box[3] - gt_box[1])

  union = pred_box_area + gt_box_area - intersection_area
  iou = intersection / union

  return 

### NMS(Non-Maximum Suppression, 비최댓값 억제)

- 확률이 가장 높은 상자와 겹치는 상자들을 제거하는 과정

- 최댓값을 갖지 않는 상자들을 제거

- 과정

  1. 확률 기준으로 모든 상자를 정렬하고 먼저 가장 확률이 높은 상자를 취함

  2. 각 상자에 대해 다른 모든 상자와의 IOU를 계산

  3. 특정 임곗값을 넘는 상자는 제거

  <img src="https://pyimagesearch.com/wp-content/uploads/2014/10/nms_fast_03.jpg">

  <sub>[이미지 출처] https://www.pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/</sub>

In [2]:
import numpy as np

In [3]:
def non_max_supression_fast(boxes, overlap_thresh):
  if len(boxes) == 0:
    return None
  if boxes.dtype.kind == 'i': #int type의 box는 float type으로 변경
    boxes = boxes.astype('float')

  pick = [] #값을 담을 빈 리스트를 하나 생성
  x1 = boxes[:, 0]
  y1 = boxes[:, 1]
  x2 = boxes[:, 2]
  y2 = boxes[:, 3]

  area = (x2 - x1 + 1) * (y2 - y1 + 1) #왜 +1 하는가?
  idxs = np.argsort(y2) #정렬 

  while len(idxs) > 0: #idx의 길이가 양수인 동안에는 
    last = len(idxs) - 1
    i = idxs[last]
    pick.append(i)

    #좌표 설정
    xx1 = np.maximum(x1[i], x1[idxs[:last]])
    yy1 = np.maximum(y1[i], y1[idxs[:last]])
    xx2 = np.maximum(x2[i], x2[idxs[:last]])
    yy2 = np.maximum(y2[i], y2[idxs[:last]])

    #여기서 왜 np.maximum 쓰는지 이해하지 못함 
    w = np.maximum(0, xx2 - xx1 + 1)
    h = np.maximum(0, yy2 - yy1 + 1)

    overlap = (w * h) / area[idxs[:last]]

    idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlap_thresh)[0])))
    #np.where로 조건 설정
    
  return boxes[pick].astype('int')

## 모델 성능 평가

### 정밀도와 재현율


- 일반적으로 객체 탐지 모델 평가에 사용되지는 않지만, 다른 지표를 계산하는 기본 지표 역할을 함

  - `TP` 

    - True Positives

    - 예측이 동일 클래스의 실제 상자와 일치하는지 측정

  - `FP`

    - False Positives

    - 예측이 실제 상자와 일치하지 않는지 측정

  - `FN`

    - False Negatives

    - 실제 분류값이 그와 일치하는 예측을 갖지 못하는지 측정

<br>

## $\qquad precision = \frac{TP}{TP \ + \ FP}$
## $\qquad recall = \frac{TP}{TP \ + \ FN}$

  - 모델이 안정적이지 않은 특징을 기반으로 객체 존재를 예측하면 거짓긍정(FP)이 많아져서 정밀도가 낮아짐

  - 모델이 너무 엄격해서 정확한 조건을 만족할 때만 객체가 탐지된 것으로 간주하면 거짓부정(FN)이 많아져서 재현율이 낮아짐



### 정밀도-재현율 곡선(precision-recall curve) 

- 신뢰도 임곗값마다 모델의 정밀도와 재현율을 시각화

- 모든 bounding box와 함께 모델이 예측의 정확성을 얼마나 확실하는지 0 ~ 1사이의 숫자로 나타내는 신뢰도를 출력

- 임계값 T에 따라 정밀도와 재현율이 달라짐

  - 임곗값 T 이하의 예측은 제거함

  - T가 1에 가까우면 정밀도는 높지만 재현율은 낮음  
    놓치는 객체가 많아져서 재현율이 낮아짐. 즉, 신뢰도가 높은 예측만 유지하기때문에 정밀도는 높아짐

  - T가 0에 가까우면 정밀도는 낮지만 재현율은 높음  
    대부분의 예측을 유지하기때문에 재현율은 높아지고, 거짓긍정(FP)이 많아져서 정밀도가 낮아짐

- 예를 들어, 모델이 보행자를 탐지하고 있으면 특별한 이유없이 차를 세우더라도 어떤 보행자도 놓치지 않도록 재현율을 높여야 함
  모델이 투자 기회를 탐지하고 있다면 일부 기회를 놓치게 되더라도 잘못된 기회에 돈을 거는 일을 피하기 위해 정밀도를 높여야 함

<img src="https://www.researchgate.net/profile/Davide_Chicco/publication/321672019/figure/fig1/AS:614279602511886@1523467078452/a-Example-of-Precision-Recall-curve-with-the-precision-score-on-the-y-axis-and-the.png">

<sub>[이미지 출처] https://www.researchgate.net/figure/a-Example-of-Precision-Recall-curve-with-the-precision-score-on-the-y-axis-and-the_fig1_321672019</sub>

### AP (Average Precision, 평균 정밀도) 와 mAP(mean Average Precision)

- 곡선의 아래 영역에 해당

- 항상 1x1 정사각형으로 구성되어 있음  
  즉, 항상 0 ~ 1 사이의 값을 가짐

- 단일 클래스에 대한 모델 성능 정보를 제공

- 전역 점수를 얻기위해서 mAP를 사용

- 예를 들어, 데이터셋이 10개의 클래스로 구성된다면 각 클래스에 대한 AP를 계산하고, 그 숫자들의 평균을 다시 구함

- (참고)

  - 최소 2개 이상의 객체를 탐지하는 대회인 PASCAL Visual Object Classes와 Common Objects in Context(COCO)에서 mAP가 사용됨

  - COCO 데이터셋이 더 많은 클래스를 포함하고 있기 때문에 보통 Pascal VOC보다 점수가 더 낮게 나옴

  - 예시

    <img src="https://www.researchgate.net/profile/Bong_Nam_Kang/publication/328939155/figure/tbl2/AS:692891936649218@1542209719916/Evaluation-on-PASCAL-VOC-2007-and-MS-COCO-test-dev.png">

    <sub>[이미지 출처] https://www.researchgate.net/figure/Evaluation-on-PASCAL-VOC-2007-and-MS-COCO-test-dev_tbl2_328939155</sub>

## Dataset

### VOC

- 2005년부터 2012년까지 진행

- Object Detection 기술의 benchmark로 간주

- 데이터셋에는 20개의 클래스가 존재

      background
      aeroplane
      bicycle
      bird
      boat
      bottle
      bus
      car
      cat
      chair
      cow
      diningtable
      dog
      horse
      motorbike
      person
      pottedplant
      sheep
      sofa
      train
      tvmonitor

- 훈련 및 검증 데이터 : 11,530개

- ROI에 대한 27,450개의 Annotation이 존재

- 이미지당 2.4개의 객체 존재

  <img src="https://paperswithcode.github.io/sotabench-eval/img/pascalvoc2012.png">

  <sub>[이미지 출처] https://paperswithcode.github.io/sotabench-eval/pascalvoc/</sub>

### COCO Dataset

- Common Objects in Context

- 200,000개의 이미지 

- 80개의 카테고리에 500,000개 이상의 객체 Annotation이 존재
      person
      bicycle
      car
      motorbike
      aeroplane
      bus
      train
      truck
      boat
      traffic light
      fire hydrant
      stop sign
      parking meter
      bench
      bird
      cat
      dog
      horse
      sheep
      cow
      elephant
      bear
      zebra
      giraffe
      backpack
      umbrella
      handbag
      tie
      suitcase
      frisbee
      skis
      snowboard
      sports ball
      kite
      baseball bat
      baseball glove
      skateboard
      surfboard
      tennis racket
      bottle
      wine glass
      cup
      fork
      knife
      spoon
      bowl
      banana
      apple
      sandwich
      orange
      broccoli
      carrot
      hot dog
      pizza
      donut
      cake
      chair
      sofa
      pottedplant
      bed
      diningtable
      toilet
      tvmonitor
      laptop
      mouse
      remote
      keyboard
      cell phone
      microwave
      oven
      toaster
      sink
      refrigerator
      book
      clock
      vase
      scissors
      teddy bear
      hair drier
      toothbrush
- https://cocodataset.org/#home

<img src="https://cocodataset.org/images/coco-examples.jpg">



# YOLO (You Only Look Once)

- 가장 빠른 객체 검출 알고리즘 중 하나

- 256x256 사이즈의 이미지

- GPU 사용 시, 초당 170프레임(170**FPS**, **frames per second**),  
  이는 파이썬, 텐서플로 기반 프레임워크가 아닌 C++로 구현된 코드 기준

- 작은 크기의 물체를 탐지하는데는 어려움

<img src="https://miro.medium.com/max/1400/1*bSLNlG7crv-p-m4LVYYk3Q.png" width="600">


- https://pjreddie.com/darknet/yolo/

- 자세한 내용 참조 : https://www.kaggle.com/aruchomu/yolo-v3-object-detection-in-tensorflow

## YOLO Backbone

- 백본 모델(backbone model) 기반

- 특징 추출기(Feature Extractor)라고도 불림

- YOLO는 자체 맞춤 아키텍쳐 사용

- 어떤 특징 추출기 아키텍쳐를 사용했는지에 따라 성능 달라짐

  <img src="https://www.researchgate.net/publication/335865923/figure/fig1/AS:804106595758082@1568725360777/Structure-detail-of-YOLOv3It-uses-Darknet-53-as-the-backbone-network-and-uses-three.jpg">

  <sub>[이미지 출처] https://www.researchgate.net/figure/Structure-detail-of-YOLOv3It-uses-Darknet-53-as-the-backbone-network-and-uses-three_fig1_335865923</sub>

- 마지막 계층은 크기가 $w \times h \times D$인 특징 볼륨 출력

- $w \times h $는 그리드의 크기이고, $D$는 특징 볼륨 깊이



*residual, upsampling, convolution 등 사용함*

## YOLO의 계층 출력

- 마지막 계층 출력은 $w \times h \times M$ 행렬
  
  - $M = B \times (C + 5)$

    - B : 그리드 셀당 경계 상자 개수

    - C : 클래스 개수

  - 클래스 개수에 5를 더한 이유는 해당 값 만큼의 숫자를 예측해야 하기 때문

    - $t_x$, $t_y$는 경계상자의 중심 좌표를 계산

    - $t_w$, $t_h$는 경계상자의 너비와 높이를 계산

    - $c$는 객체가 경계 상자 안에 있다고 확신하는 신뢰도

    - $p1, p2, ..., pC$는 경계상자가 클래스 1, 2, ..., C의 객체를 포함할 확률
  
  <br>

  <img src="https://www.researchgate.net/profile/Thi_Le3/publication/337705605/figure/fig3/AS:831927326089217@1575358339500/Structure-of-one-output-cell-in-YOLO.ppm">

  <sub>[이미지 출처] https://www.researchgate.net/figure/Structure-of-one-output-cell-in-YOLO_fig3_337705605</sub>

## 앵커 박스(Anchor Box)

- YOLOv2에서 도입

- 사전 정의된 상자(prior box)

- 객체에 가장 근접한 앵커 박스를 맞추고 신경망을 사용해 앵커 박스의 크기를 조정하는 과정 때문에 $t_x, t_y, t_w, t_h$이 필요

  <img src="https://kr.mathworks.com/help/vision/ug/ssd_detection.png">

  <sub>[이미지 출처] https://kr.mathworks.com/help/vision/ug/getting-started-with-yolo-v2.html</sub>

# YOLOv3 Inference 연습 : tensorflow 2

- 코드 참조 : https://github.com/zzh8829/yolov3-tf2

## Clone and install dependencies

In [4]:
!git clone https://github.com/zzh8829/yolov3-tf2 
%cd yolov3-tf2
!pip install -r requirements-gpu.txt

fatal: destination path 'yolov3-tf2' already exists and is not an empty directory.
/content/yolov3-tf2
Obtaining file:///content/yolov3-tf2 (from -r requirements-gpu.txt (line 6))
  Using cached numpy-1.19.5-cp37-cp37m-manylinux2010_x86_64.whl (14.8 MB)
  Using cached keras_nightly-2.5.0.dev2021032900-py2.py3-none-any.whl (1.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: yolov3-tf2
    Found existing installation: yolov3-tf2 0.1
    Can't uninstall 'yolov3-tf2'. No files were found to uninstall.
  Running setup.py develop for yolov3-tf2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have foli

In [5]:
!ls #설치 후 확인

checkpoints	 data		  README.md		train.py
colab_gpu.ipynb  detect.py	  requirements-gpu.txt	yolov3_tf2
conda-cpu.yml	 detect_video.py  requirements.txt	yolov3_tf2.egg-info
conda-gpu.yml	 docs		  setup.py
convert.py	 LICENSE	  tools


## Check Tensorflow2 version

In [ ]:
!pip uninstall -y tensorflow keras tf-nightly keras-nightly
!pip install tensorflow

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Successfully uninstalled tensorflow-2.8.0
Found existing installation: keras 2.8.0
Uninstalling keras-2.8.0:
  Successfully uninstalled keras-2.8.0
Found existing installation: keras-nightly 2.5.0.dev2021032900
Uninstalling keras-nightly-2.5.0.dev2021032900:
  Successfully uninstalled keras-nightly-2.5.0.dev2021032900


In [ ]:
import tensorflow as tf
tf.__version__

## Convert Pretrained Darknet Weight

- https://pjreddie.com/media/files/yolov3.weights
- `yolov3.weights`를 `/yolov3-tf2/data/`로 넣기

In [ ]:
!get https://pjreddie.com/media/files/yolov3.weights -0 data/yolov3.weights
!python convert.py

## Initial Detector

In [ ]:
import sys
from absl import app, logging, flags #1이 아니고 l임
from absl.flags import FLAGS
import time
import cv2
import numpy as np
import tensorflow as tf
from yolov3_tf2.models import YoloV3, YoloV3Tiny
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs

In [ ]:
flags.DEFINE_string('classes', './data/coco.names', 'path to classes file')
flags.DEFINE_string('weight', './checkpoints/yolov3.tf', 'path to wieghts file')
flags.DEFINE_boolean('tiny', False, 'yolov3 or yolov3-tiny')
flags.DEFINE_integer('size', 416, 'resize images to')
flags.DEFINE_string('image', './data/girl.png', 'path to input image')
flags.DEFINE_string('tfrecord', None, 'tfrecord instead of image')
flags.DEFINE_string('output', './output.jpg', 'path to output image')
flags.DEFINE_integer('num_classes', 80, 'number of classes in the model')

In [ ]:
app._run_init(['yolov3']. app.parse_flags_with_usage)

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Detect Image

In [ ]:
FLAGS.image = 'data/mem.jpg'

In [ ]:
if FLAGS.tiny:
  yolo = YoloV3Tiny(classes = FLAGS.num_classes)
else:
  yolo = YoloV3(classese=FLAGS.num_classes)

yolo.load_weights(FLAGS.weights).expect_partial()
logging.info('weights loaded') #로그인 기록

class_names = [c.strip() for c in open(FLAGS.classes).readlines()]
logging.info('classes loaded')

In [ ]:
img_raw = tf.image.decode_image(open(FLAGS.image, 'rb').read(), channels = 3)
img = tf.expand_dims(img_raw, 0)
img = transform_images(img, FLAGS.size)

t1 = time.time()
boxes, scores, classes, nums = yolo(img)
t2 = time.time()
logging.info('time:{}'.format(t2 - t1))

logging.info('detections: ')
for i in range(nums[0]):
  logging.info('  {}, {}, {}'.format(class_names[int(classes[0][i])], 
                                     np.array(scores[0][i]),
                                     np.array(boxes[0][i])))
img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)
img = draw_outputs(img, (boxes, scores, classes, nums), class_names)

In [ ]:
from IPython.display import Image, display

display(Image(data=bytes(cv2.imencode('.jpg', img)[1]), width=800))

# YOLOv3 inference 연습 : Pytorch 

- video inference
- 코드 참조 : https://towardsdatascience.com/yolov3-pytorch-on-google-colab-c4a79eeecdea
- https://github.com/ultralytics/yolov3

## Prepare YOLOv3-pytorch

In [ ]:
%cd /content/
%pwd

In [ ]:
!git clone https://github.com/ultralytics/yolov3

In [ ]:
%cd yolov3

In [ ]:
import time
import glob
import torch
import os

import argparse
from sys import platform

from models import *
from utils.datasets import *
from utils.torch_utils import *

from IPython.display import HTML #비디오 출력을 위해
from base64 import b64encode

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--cfg', type=str, default='cfg/yolov3-spp.cfg', help = '*.cfg path')
parser.add_argument('--names', type=str, default='data/coco.names', help = '*.names path')
parser.add_argument('--weights', type=str, default='weights/yolov3-spp-ultralytics.pt', help = 'weights path')
parser.add_argument('--img-size', type=int, default=416, help = 'inference size (pixels)')
parser.add_argument('--conf-thres', type=float, default=0.3, help = 'object confidence threshold')
parser.add_argument('--iou-thres', type=float, default=0.6, help = 'IOU threshold for NMS')
parser.add_argument('--device', default='', help = 'device id (i.e. 0 or 0, 1) or cpu')
parser.add_argument('--classes', nargs = '+', type = int, help = 'filter by class')
parser.add_argument('--agnostic-mns', action='store_true', help='class0agnostic NMS')
opt = parser.parse_args(args = [])

weights = opt.weights
img_size = opt.img_size

In [ ]:
device = torch_utils.select_device(device = 'cpu' if ONIX_EXPORT else opt.device)
model = Darknet(opt.cfg, img_size)
attemp_download(weights)
if weights.endswith('.pt'):
  model.load_state_dict(torch.load(weights, map_location = device)['model'])
else:
  load_darknet_weights(model, weights)

model.to(device).eval();

names = load_classes(opt.names)
colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]

In [ ]:
%cd ..

In [ ]:
def predict_one_video(path_video, output_dir='output'):
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)

  cap = cv2.VideoCapture(path_video)
  _, img0 = cap.read()

  save_path = os.path.join(output_dir, os.path.split(path_video)[-1])
  fps = cap.get(cv2.CAP_PROP_FPS)
  w = int(cap.get(cv2.CAP_FRAME_WIDTH))
  h = int(cap.get(cv2.CAP_FRAME_HEIGHT))
  vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'MP4V'), fps, (w, h))

  while img0 is not None:
    img = letterbox(img0, new_shape = opt.img.size)[0]

    img = img[:, :, ::-1].transpose(2, 0, 1)
    img = np.ascontinguousarry(img)

    img = torch.from_numpy(img).to(device)
    img = img.float()
    img /= 255.0

    if img.ndimension() == 3:
      img  = img.unsqueeze(0)

    pred = model(img)[0]

    pred = non_max_suppression(pred, opt.conf_thres, opt.iou_thres, classes = opt.classes, agnostic = opt.agnostic_nms)

    #detection
    for i, det in enumerate(pred):
      im0 = img0 #이거 왜하는거임 
      
      if det is not None and len(det):
        det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round() #box rescale

        for *xyxy, conf, cls in det:
          label = '%s %.2f'% (names[int(cls)], conf)
          plot_one_box(xyxy, im0, label = label, color = colors[int(cls)])

    vid_writer.write(im0)
    _, img0 - cap.read()

  vid_writer.release()

  return save_path
        

## Git clone to get short videos

- https://github.com/vindruid/yolov3-in-colab.git

In [ ]:
!git clone https://github.com/vindruid/yolov3-in-colab.git
!cp -r 'yolov3-in-colab'/input_video/* ./input_video/

## Process Video

In [ ]:
!mkdir -p input_video
!mkdir -p output_compressed

In [ ]:
path_video = os.path.join('/content/yolov3-in-colab/input_video', 'opera_house.mp4')
save_path = predict_one_video(path_video)

In [ ]:
print(path_video)

In [ ]:
mp4 = open(path_video, 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML("""
<video width = 400 controls>
  <sounce src='%s' type = 'video/mp4'>
</video>
  """ % data_url)

In [ ]:
#detection 결과 

compressed_path = os.path.join('output_compressed', os.path.split(save_path)[-1])
os.system(f'ffmpeg -i {save_path} -vcodec libx264 {compressed_path}')

mp4 = open(path_video, 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML("""
<video width = 400 controls>
  <sounce src='%s' type = 'video/mp4'>
</video>
  """ % data_url)

- Custom Data Train 참고

  - https://github.com/AntonMu/TrainYourOwnYOLO